# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold, train_test_split, GroupKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, precision_recall_fscore_support
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LSTM, SimpleRNN, GRU, ConvLSTM2D, Bidirectional, TimeDistributed, GaussianNoise, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model, to_categorical
from torch.utils.data import DataLoader, TensorDataset
import os
import sys
import time
import csv

"    \n    'F1': ['d1', 'd3', '12-x', '12-y', '16-x', '16-y', '14-x', '14-y', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],\n    'F2': ['d1', 'd3', 'd5', 'd7', '16-x', '16-y', '14-x', '14-y', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],   \n    'F3': ['d1', 'd3', '12-x', '12-y', '16-x', '16-y', '14-x', '14-y', 'd5', 'd7', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],\n    'F4': ['d1', 'd3', '12-x', '12-y', '16-x', '16-y', '14-x', '14-y', '11-x', '11-y', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],\n    'F5': ['11-x', '11-y', '12-x', '12-y', '13-x', '13-y', '14-x', '14-y', '15-x', '15-y', '16-x', '16-y', '0-x', '0-y', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z']\n\n  "

# DataToImage

In [ ]:

def read_data(file_name):
    
    df = pd.read_csv(file_name, sep='\t')
    return df


# Sliding windows on the timestamps 

def rolling_window(fileName, window_size,freq): # window size is specified by user, freq means the overlap
    array = np.array(fileName.astype(float))
    #array = float(np.nan_to_num(array)) #put 0 in nan place
    shape = (array.shape[0] - window_size + 1, window_size)
    strides = (array.strides[0],) + array.strides
    rolled = np.lib.stride_tricks.as_strided(array, shape=shape, strides=strides)
    return rolled[np.arange(0,shape[0],freq)]



def siplite_scaleFiles(dataframe, window, stride, columns):
    scaler = MinMaxScaler(feature_range=(0, 255))
    results = {}

    # Iterate over the columns and assign each to a letter starting from 'a'
    for idx, column in enumerate(columns):
        letter = chr(ord('a') + idx)  # Generate letters 'a', 'b', 'c', etc.
        scaled_data = scaler.fit_transform(rolling_window(dataframe[column].values, window, stride))
        results[letter] = scaled_data  # Store the result in a dictionary with the letter as key

    # Return the values of the dictionary in the order of the letters
    return tuple(results[chr(ord('a') + i)] for i in range(len(columns)))

def feature_sets(scaled_values,bundle):

    if bundle == "F0" or bundle == "F1" or bundle == "F2":
        # Assign variables dynamically using unpacking
        a, b, c, d, e, f, g, h, i, j, k = scaled_values

        # Vision feature stacking (using the order specified)
        vision = np.hstack((a, b, c, d, e, f, g, h, a, c, e, g, a, d, f, h, b, d, g, b, e, h, c, f, a, e))

        # IMU feature stacking
        imu = np.hstack((i, j, k, i, j, k, i, j, k, i, j, k, i))
        
        nvision = 26

        nimu = 13
    elif bundle == "F3" or bundle == "F4":
        # Assign variables dynamically using unpacking
        a, b, c, d, e, f, g, h, i, j,s,p,n = scaled_values
        print(f"Executing block for {bundle}")
        # Vision feature stacking (using the order specified)
        vision = np.hstack((a,b,c,d,e,f,g,h,i,j,a,c,e,g,i,a,d,f,h,j,b,d,g,j,c,f,i,b,e,h,a,e,i,c,g,a,f,j,d,h,b,f))

        # IMU feature stacking
        imu = np.hstack((s,p,n,s,p,n,s,p,n,s,p,n,s))
        
        nvision = 42

        nimu = 13

    elif bundle == "F5":
        # Assign variables dynamically using unpacking
        a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y = scaled_values

        # Vision feature stacking (using the order specified)
        vision = np.hstack((a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,a,c,e,g,i,k,m,o,q,s,u,a,d,f,h,j,l,n,p,r,t,v,b,d,g,j,m,p,s,v,c,f,i,l,o,r,u,b,e,h,k,n,q,t,a,e,i,m,q,u,c,g,k,o,s,a,f,j,n,r,v,d,h,l,p,t,b,f,k,p,u,d,i,n,s,b,g,l,q,v,e,j,o,t,c,h,m,r,a,g,m,s,c,i,o,u,e,k,q,a,h,n,t,d,j,p,v,f,l,r,b,h,o,v,g,n,u,f,m,t,e,l,s,d,k,r,c,j,q,b,i,p,a,i,q,c,k,s,e,m,u,g,o,a,j,r,d,l,t,f,n,v,h,p,b,j,s,f,o,b,k,t,g,p,c,l,u,h,q,d,m,v,i,r,e,n,a,k,u,i,s,g,q,e,o,c,m,a,l,v,j,t,h,r,f,p,d,n,b,l))

        # IMU feature stacking
        imu = np.hstack((w,x,y,w,x,y,w,x,y,w,x,y,w)) 
        
        nvision = 222

        nimu = 13
    return vision, imu , nvision, nimu

def stack_Data(dataframe, window, stride, columns,bundle):
    subject_column = dataframe['subject'].values
    unique_subjects = np.unique(subject_column)
    
    all_stacked1 = []
    all_stacked2 = []
    all_subjects = []
    
    for sub in unique_subjects:
        sub_dataframe = dataframe[dataframe['subject'] == sub]

        # Dynamically assign the letters from siplite_scaleFiles output
        scaled_values = siplite_scaleFiles(sub_dataframe, window, stride, columns)
        
        # Get vision and imu feature sets
        vision, imu , nvision, nimu = feature_sets(scaled_values,bundle)

        # Stack sensor data channels
        h1 = vision
        i1 = imu

        h1 = h1.reshape((h1.shape[0], nvision, window)) # convert sensor data to image  26
        i1 = i1.reshape((i1.shape[0], nimu, window)) # 13

        all_stacked1.append(h1)
        all_stacked2.append(i1) 

        # Append the subject 'sub' for each row in h1
        all_subjects.append(np.full((h1.shape[0], 1), sub)) 
         


    return np.vstack(all_stacked1), np.vstack(all_stacked2), np.vstack(all_subjects)


def DataToImage(WindowSize,Overlapping,columns,bundle):

    ex11_v , ex11_i , subject11 = stack_Data(read_data('data/11.txt'), WindowSize, Overlapping ,columns,bundle)
    ex12_v , ex12_i , subject12= stack_Data(read_data('data/12.txt'), WindowSize, Overlapping ,columns,bundle)
    ex13_v , ex13_i , subject13= stack_Data(read_data('data/13.txt'), WindowSize, Overlapping ,columns,bundle)
    ex21_v , ex21_i , subject21= stack_Data(read_data('data/21.txt'), WindowSize, Overlapping ,columns,bundle)
    ex22_v , ex22_i , subject22= stack_Data(read_data('data/22.txt'), WindowSize, Overlapping ,columns,bundle)
    ex23_v , ex23_i , subject23= stack_Data(read_data('data/23.txt'), WindowSize, Overlapping ,columns,bundle)
    ex31_v , ex31_i , subject31= stack_Data(read_data('data/31.txt'), WindowSize,Overlapping ,columns,bundle)
    ex32_v , ex32_i , subject32= stack_Data(read_data('data/32.txt'), WindowSize, Overlapping ,columns,bundle)
    ex33_v , ex33_i , subject33= stack_Data(read_data('data/33.txt'), WindowSize, Overlapping ,columns,bundle)
    ex41_v , ex41_i , subject41= stack_Data(read_data('data/41.txt'), WindowSize, Overlapping ,columns,bundle)
    ex42_v , ex42_i , subject42= stack_Data(read_data('data/42.txt'), WindowSize, Overlapping ,columns,bundle)
    ex43_v , ex43_i , subject43= stack_Data(read_data('data/43.txt'), WindowSize, Overlapping ,columns,bundle)
    #stack all images datasets in one set
    data_v = np.concatenate((ex11_v,ex12_v,ex13_v,ex21_v,ex22_v,ex23_v,ex31_v,ex32_v,ex33_v,ex41_v,ex42_v,ex43_v), axis=0)
    data_i = np.concatenate((ex11_i,ex12_i,ex13_i,ex21_i,ex22_i,ex23_i,ex31_i,ex32_i,ex33_i,ex41_i,ex42_i,ex43_i), axis=0)
    subject_all = np.concatenate((subject11,subject12,subject13,subject21,subject22,subject23,subject31,subject32,subject33,subject41,subject42,subject43), axis=0)
    
    print(subject_all.shape)
    print(data_i.shape)
    #generate labels for the target images
    y1 = np.zeros([ex11_v.shape[0]], dtype = np.uint8)
    y2 = np.zeros([ex12_v.shape[0]], dtype = np.uint8)
    y3 = np.zeros([ex13_v.shape[0]], dtype = np.uint8)
    y4 = np.zeros([ex21_v.shape[0]], dtype = np.uint8)
    y5 = np.zeros([ex22_v.shape[0]], dtype = np.uint8)
    y6 = np.zeros([ex23_v.shape[0]], dtype = np.uint8)
    y7 = np.zeros([ex31_v.shape[0]], dtype = np.uint8)
    y8 = np.zeros([ex32_v.shape[0]], dtype = np.uint8)
    y9 = np.zeros([ex33_v.shape[0]], dtype = np.uint8)
    y10= np.zeros([ex41_v.shape[0]], dtype = np.uint8)
    y11= np.zeros([ex42_v.shape[0]], dtype = np.uint8)
    y12= np.zeros([ex43_v.shape[0]], dtype = np.uint8)
    y1[:] = 1
    y2[:] = 1
    y3[:] = 1
    y4[:] = 2
    y5[:] = 2
    y6[:] = 2
    y7[:] = 3
    y8[:] = 3
    y9[:] = 3
    y10[:] = 4
    y11[:] = 4
    y12[:] = 4
    #target = np.vstack((y1,y2,y3,y4,y5,y6))
    target = np.concatenate((y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12), axis=0)
    #target = np.concatenate((y1,y2,y3,y4,y5), axis=0)
    
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(target)
    encoded_Y = encoder.transform(target)   
    #convert integers to dummy variables (one hot encoded)
    
    Y = np_utils.to_categorical(encoded_Y)
    return data_v, target, Y ,data_i , subject_all


# List of bundles and corresponding column sets
bundle_column_mapping = {

    'F0': ['d1', 'd3', 'd5', 'd7', 'd2', 'd4', 'd6', 'd8', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],
    'F1': ['d1', 'd3', '12-x', '12-y', '16-x', '16-y', '14-x', '14-y', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],
    'F2': ['d1', 'd3', 'd5', 'd7', '16-x', '16-y', '14-x', '14-y', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],   
    'F3': ['d1', 'd3', '12-x', '12-y', '16-x', '16-y', '14-x', '14-y', 'd5', 'd7', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],
    'F4': ['d1', 'd3', '12-x', '12-y', '16-x', '16-y', '14-x', '14-y', '11-x', '11-y', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z'],
    'F5': ['11-x', '11-y', '12-x', '12-y', '13-x', '13-y', '14-x', '14-y', '15-x', '15-y', '16-x', '16-y', '0-x', '0-y', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z']


}

WindowSize = 60      
Overlapping = 15     # Adjust this based on your requirements



# Computed Features

In [ ]:


# Function to read individual data file and assign class labels
def read_data1(file_name, file_class_map):
    print(f"Loading data from {file_name}")
    df1 = pd.read_csv(file_name, sep='\t')
    base_name = file_name.split('/')[-1].split('.')[0]
    df1['class'] = file_class_map[base_name]
    return df1


# Function to load data from a directory and apply class mapping
def load_data1(directory, files, file_class_map):
    data_frames1 = []
    for file_name in files:
        file_path = f'{directory}/{file_name}.txt'
        df1 = read_data1(file_path, file_class_map)
        data_frames1.append(df1)
    return pd.concat(data_frames1, ignore_index=True)


# Function to extract features using a sliding window approach
def extract_features_and_labels(data1, feature_names, window_size, overlap):
    feature_data = data1[feature_names]
    subjects = data1['subject'].values
    labels = data1['class'].values


    n_samples = feature_data.shape[0]
    step = window_size - overlap
    extracted_features = []
    extracted_labels = []
    extracted_subjects = []  # Store subjects for group k-fold


    for start in range(0, n_samples - window_size + 1, step):
        end = start + window_size
        if len(np.unique(subjects[start:end])) == 1:
            window = feature_data.iloc[start:end].values
            mean = np.mean(window, axis=0)
            std = np.std(window, axis=0)
            min_val = np.min(window, axis=0)
            max_val = np.max(window, axis=0)
            extracted_features.append(np.concatenate([mean, std, min_val, max_val]))
            extracted_labels.append(labels[start])
            extracted_subjects.append(subjects[start])


    return np.array(extracted_features), np.array(extracted_labels), np.array(extracted_subjects)



# File names and directories
file_names = ['11', '12', '13', '21', '22', '23', '31', '32', '33', '41', '42', '43']
data_directory = 'data'


# Class mapping
file_class_map = {
    '11': 1, '12': 1, '13': 1,
    '21': 2, '22': 2, '23': 2,
    '31': 3, '32': 3, '33': 3,
    '41': 4, '42': 4, '43': 4
}


all_data = load_data1(data_directory, file_names, file_class_map)




Loading data from data/11.txt
Loading data from data/12.txt
Loading data from data/13.txt
Loading data from data/21.txt
Loading data from data/22.txt
Loading data from data/23.txt
Loading data from data/31.txt
Loading data from data/32.txt
Loading data from data/33.txt
Loading data from data/41.txt
Loading data from data/42.txt
Loading data from data/43.txt


# CNN+CF

In [ ]:
name = "subclass-featurefusion-newfeatures-4class-noise"


# Check if GPU is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class CNNWithRF(nn.Module):
    def __init__(self, input_shape_v, input_shape_i, num_classes, num_rf_features):
        super(CNNWithRF, self).__init__()

        # CNN for vision data
        self.cnn_v = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )

        # CNN for IMU data
        self.cnn_i = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )

        # Calculate CNN output size dynamically
        conv_out_v = self._conv_output_size(input_shape_v)
        conv_out_i = self._conv_output_size(input_shape_i)

        # Dense layers after concatenation
        self.fc = nn.Sequential(
            nn.Linear(conv_out_v + conv_out_i + num_rf_features, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def _conv_output_size(self, input_shape):
        dummy_input = torch.zeros(1, *input_shape)  
        output = self.cnn_v(dummy_input)
        return int(np.prod(output.size()))

    def forward(self, x_v, x_i, rf_features):
        out_v = self.cnn_v(x_v)
        out_i = self.cnn_i(x_i)
        combined = torch.cat((out_v, out_i, rf_features), dim=1)
        output = self.fc(combined)
        return output


# Train the model for a single fold
def train_model(model, optimizer, criterion, dataloader_train, dataloader_val, epochs=30, fold=None, bundle=None):
    results = {"train_loss": [], "train_accuracy": [], "val_loss": [], "val_accuracy": []}

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for X_v, X_i, features, y in dataloader_train:
            # Move data to the GPU
            X_v, X_i, features, y = X_v.to(device), X_i.to(device), features.to(device), y.to(device)

            optimizer.zero_grad()
            outputs = model(X_v, X_i, features)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == y).sum().item()
            total_train += y.size(0)

        train_loss = running_loss / len(dataloader_train)
        train_accuracy = correct_train / total_train

        # Validation step
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for X_v, X_i, features, y in dataloader_val:
                # Move data to the GPU
                X_v, X_i, features, y = X_v.to(device), X_i.to(device), features.to(device), y.to(device)

                outputs = model(X_v, X_i, features)
                loss = criterion(outputs, y)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == y).sum().item()
                total_val += y.size(0)

        val_loss /= len(dataloader_val)
        val_accuracy = correct_val / total_val

        results["train_loss"].append(train_loss)
        results["train_accuracy"].append(train_accuracy)
        results["val_loss"].append(val_loss)
        results["val_accuracy"].append(val_accuracy)

        print(f"Epoch [{epoch + 1}/{epochs}] - Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    return results


# Evaluate the model
def evaluate_model(model, dataloader_test, criterion):
    model.eval()
    all_preds = []
    all_true = []
    total_loss = 0.0
    with torch.no_grad():
        for X_v, X_i, features, y in dataloader_test:
            # Move data to the GPU
            X_v, X_i, features, y = X_v.to(device), X_i.to(device), features.to(device), y.to(device)

            outputs = model(X_v, X_i, features)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_preds.append(preds.cpu().numpy())
            all_true.append(y.cpu().numpy())
    
    all_preds = np.concatenate(all_preds)
    all_true = np.concatenate(all_true)
    accuracy = accuracy_score(all_true, all_preds)
    precision = precision_score(all_true, all_preds, average='weighted')
    f1 = f1_score(all_true, all_preds, average='weighted')
    recall = recall_score(all_true, all_preds, average='weighted')
    cm = confusion_matrix(all_true, all_preds)

    avg_loss = total_loss / len(dataloader_test)

    return accuracy, precision, f1, recall, cm, avg_loss

# Main loop with cross-validation
for bundle, columns in bundle_column_mapping.items():
    print(f"\nProcessing bundle: {bundle}, Columns: {columns}")

    # Load data (vision, imu, rf features, and labels)
    extracted_features, labels, subjects_extracted = extract_features_and_labels(all_data, columns, window_size=60, overlap=45)
    x_v, target, y, x_i, subjects = DataToImage(WindowSize, Overlapping, columns, bundle)


    #apply noise
    #noise_extracted_features=rf_noise_dist(extracted_features)
    #noise_x_v=v_noise_dist(x_v)
    #noise_x_i=i_noise_dist(x_i)
    
    #extracted_features=noise_extracted_features
    #x_v=noise_x_v
    #x_i=noise_x_i
    # Reshape subjects to be a 1D array (9744,)
    subjects = subjects.ravel()

    # Convert data to torch tensors
    x_v = torch.tensor(x_v, dtype=torch.float32)
    x_i = torch.tensor(x_i, dtype=torch.float32)
    y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)
    extracted_features = torch.tensor(extracted_features, dtype=torch.float32)

    # Add the channel dimension to the input tensors
    x_v = x_v.unsqueeze(1)
    x_i = x_i.unsqueeze(1)

    # Lists to store fold-wise test metrics
    all_test_accuracy = []
    all_test_precisions = []
    all_test_f1_scores = []
    all_test_recalls = []
    all_test_loss = []

    # KFold cross-validation
    unique_subjects = np.unique(subjects)

    for test_subject in unique_subjects:
        # Create train and test masks
        test_mask = subjects == test_subject
        train_mask = subjects != test_subject

        # Split the data based on the masks
        trainx_v, testX_v = x_v[train_mask], x_v[test_mask]
        trainx_i, testX_i = x_i[train_mask], x_i[test_mask]
        trainy, testY = y[train_mask], y[test_mask]
        train_subjects, test_subjects = subjects[train_mask], subjects[test_mask]
        tfeature, test_feature = extracted_features[train_mask], extracted_features[test_mask]
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for fold, (train_index, val_index) in enumerate(kf.split(trainx_v)):
            X_train_v, X_val_v = trainx_v[train_index], trainx_v[val_index]
            X_train_i, X_val_i = trainx_i[train_index], trainx_i[val_index]
            y_train, y_val = trainy[train_index], trainy[val_index]
            train_features, val_features = tfeature[train_index], tfeature[val_index]

            # Create PyTorch DataLoader for training and validation
            train_data = TensorDataset(X_train_v, X_train_i, train_features, y_train)
            val_data = TensorDataset(X_val_v, X_val_i, val_features, y_val)

            train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
            val_loader = DataLoader(val_data, batch_size=16, shuffle=False)

            # Define model, optimizer, and loss function
            input_shape_v = (1,x_v.shape[2], x_v.shape[3])
            input_shape_i = (1,x_i.shape[2], x_i.shape[3])
            num_classes = len(np.unique(y.numpy()))

            # Move the model to GPU
            model = CNNWithRF(input_shape_v, input_shape_i, num_classes, num_rf_features=train_features.shape[1])
            model.to(device)  # Move the model to GPU

            # Use GPU for optimization and loss calculation
            optimizer = optim.Adam(model.parameters(), lr=0.00031479)
            criterion = nn.CrossEntropyLoss().to(device)  # Move the loss function to GPU

            # Train the model and save metrics
            train_model(model, optimizer, criterion, train_loader, val_loader, epochs=30, fold=fold, bundle=bundle)

            # Evaluate the model on the validation set
            accuracy, precision, f1, recall, cm, loss = evaluate_model(model, val_loader, criterion)

            print(f"Fold {fold} - Accuracy:{accuracy} , Precision: {precision}, F1: {f1}, Recall: {recall}, Loss: {loss}")
            print(f"Confusion Matrix:\n{cm}")

            # Save model, confusion matrix, and metrics
            torch.save(model.state_dict(), f'save/model/{name}_{bundle}_fold_{fold}_{test_subject}.pt')

            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
            plt.title(f"Confusion Matrix - Fold {fold}")
            plt.savefig(f'save/confusion_matrices/{name}_{bundle}_fold_{fold}_{test_subject}.png')
            plt.close()

                        # Now evaluate on the test set
            test_loader = DataLoader(TensorDataset(torch.tensor(testX_v), torch.tensor(testX_i), torch.tensor(test_feature), torch.tensor(testY)),
                                     batch_size=1, shuffle=False)
            accuracy_test, precision_test, f1_test, recall_test, cm_test , test_loss = evaluate_model(model, test_loader, criterion)
            print(f"bundle: {bundle}\n")
            print(f"Test Set Evaluation - Fold {fold}")
            print(f"Test Accuracy: {accuracy_test} , Precision: {precision_test}, F1: {f1_test}, Test Recall: {recall_test}, error: {test_loss}")
            print(f"Test Confusion Matrix:\n{cm_test}")
            
            # Save the test metrics for later averaging
            all_test_loss.append(test_loss)
            all_test_accuracy.append(accuracy_test)
            all_test_precisions.append(precision_test)
            all_test_f1_scores.append(f1_test)
            all_test_recalls.append(recall_test)
            all_test_recalls.append(recall_test)
            all_test_loss.append(test_loss)
            # Save the test results in a separate text file
            test_results_filename = f'save/test_results/{name}_{bundle}_test_fold_{fold}_{test_subject}.txt'
            os.makedirs(os.path.dirname(test_results_filename), exist_ok=True)
            with open(test_results_filename, 'w') as f:
                #f.write(f"Test Loss: {loss_test}\n")
                f.write(f"Test Accuracy: {accuracy_test}\n")
                f.write(f"Test Precision: {precision_test}\n")
                f.write(f"Test F1 Score: {f1_test}\n")
                f.write(f"Test Recall: {recall_test}\n")
                f.write(f"Test Recall: {all_test_loss}\n")
                f.write(f"Test Confusion Matrix:\n{cm_test}\n")

            # Save the test confusion matrix
            sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues")
            plt.title(f"Confusion Matrix - Test Set - Fold {fold}")
            plt.savefig(f'save/confusion_matrices/{name}_{bundle}_test_fold_{fold}_{test_subject}.png')
            plt.close()
    # Calculate mean of test metrics across all folds
    mean_accuracy = np.mean(all_test_accuracy)
    mean_precision = np.mean(all_test_precisions)
    mean_f1_score = np.mean(all_test_f1_scores)
    mean_recall = np.mean(all_test_recalls)
    mean_loss = np.mean(all_test_loss)
    # Save the mean test results to a file
    mean_test_results_filename = f'save/test_results/{name}_{bundle}_mean_test_results.txt'
    with open(mean_test_results_filename, 'w') as f:
        f.write(f"Mean Test Acccuracy: {mean_accuracy}\n")
        f.write(f"Mean Test Precision: {mean_precision}\n")
        f.write(f"Mean Test F1 Score: {mean_f1_score}\n")
        f.write(f"Mean Test Recall: {mean_recall}\n")
        f.write(f"Mean Test Loss: {mean_loss}\n")


Processing bundle: F0, Columns: ['d1', 'd3', 'd5', 'd7', 'd2', 'd4', 'd6', 'd8', 'Raw-accel-x', 'Raw-accel-y', 'Raw-accel-z']
(9744, 1)
(9744, 13, 60)
Epoch [1/30] - Train Loss: 0.7320, Train Accuracy: 0.9302, Val Loss: 0.0637, Val Accuracy: 0.9785
Epoch [2/30] - Train Loss: 0.0532, Train Accuracy: 0.9849, Val Loss: 0.0391, Val Accuracy: 0.9857
Epoch [3/30] - Train Loss: 0.0375, Train Accuracy: 0.9902, Val Loss: 0.0237, Val Accuracy: 0.9922
Epoch [4/30] - Train Loss: 0.0161, Train Accuracy: 0.9953, Val Loss: 0.0353, Val Accuracy: 0.9883
Epoch [5/30] - Train Loss: 0.0182, Train Accuracy: 0.9933, Val Loss: 0.0660, Val Accuracy: 0.9805
Epoch [6/30] - Train Loss: 0.0360, Train Accuracy: 0.9888, Val Loss: 0.0393, Val Accuracy: 0.9870
Epoch [7/30] - Train Loss: 0.0105, Train Accuracy: 0.9966, Val Loss: 0.0512, Val Accuracy: 0.9844
Epoch [8/30] - Train Loss: 0.0134, Train Accuracy: 0.9951, Val Loss: 0.0686, Val Accuracy: 0.9792
Epoch [9/30] - Train Loss: 0.0259, Train Accuracy: 0.9915, Val L

/tmp/ipykernel_25415/3627583972.py:264: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_loader = DataLoader(TensorDataset(torch.tensor(testX_v), torch.tensor(testX_i), torch.tensor(test_feature), torch.tensor(testY)),


bundle: F0

Test Set Evaluation - Fold 0
Test Accuracy: 0.9854085603112841 , Precision: 0.9855064632870929, F1: 0.9853854240589787, Test Recall: 0.9854085603112841, error: 0.08522740250795426
Test Confusion Matrix:
[[467   0   0   0]
 [  0 482  14   3]
 [  0   0 511   0]
 [  1  12   0 566]]
Epoch [1/30] - Train Loss: 1.2525, Train Accuracy: 0.9031, Val Loss: 0.0839, Val Accuracy: 0.9766
Epoch [2/30] - Train Loss: 0.0595, Train Accuracy: 0.9829, Val Loss: 0.0464, Val Accuracy: 0.9870
Epoch [3/30] - Train Loss: 0.0383, Train Accuracy: 0.9891, Val Loss: 0.0398, Val Accuracy: 0.9902
Epoch [4/30] - Train Loss: 0.0293, Train Accuracy: 0.9914, Val Loss: 0.0489, Val Accuracy: 0.9818
Epoch [5/30] - Train Loss: 0.0180, Train Accuracy: 0.9951, Val Loss: 0.0488, Val Accuracy: 0.9863
Epoch [6/30] - Train Loss: 0.0175, Train Accuracy: 0.9950, Val Loss: 0.0675, Val Accuracy: 0.9876
Epoch [7/30] - Train Loss: 0.0054, Train Accuracy: 0.9984, Val Loss: 0.0307, Val Accuracy: 0.9922
Epoch [8/30] - Train L

/tmp/ipykernel_25415/3627583972.py:264: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_loader = DataLoader(TensorDataset(torch.tensor(testX_v), torch.tensor(testX_i), torch.tensor(test_feature), torch.tensor(testY)),


bundle: F0

Test Set Evaluation - Fold 1
Test Accuracy: 0.9883268482490273 , Precision: 0.9883949173934318, F1: 0.9883233216696509, Test Recall: 0.9883268482490273, error: 0.047071101942093525
Test Confusion Matrix:
[[460   0   3   4]
 [  0 492   5   2]
 [  0   0 511   0]
 [  3   4   3 569]]
Epoch [1/30] - Train Loss: 0.6518, Train Accuracy: 0.9148, Val Loss: 0.0672, Val Accuracy: 0.9811
Epoch [2/30] - Train Loss: 0.0439, Train Accuracy: 0.9873, Val Loss: 0.0725, Val Accuracy: 0.9831
Epoch [3/30] - Train Loss: 0.0212, Train Accuracy: 0.9935, Val Loss: 0.0439, Val Accuracy: 0.9863
Epoch [4/30] - Train Loss: 0.0195, Train Accuracy: 0.9937, Val Loss: 0.0549, Val Accuracy: 0.9850
Epoch [5/30] - Train Loss: 0.0220, Train Accuracy: 0.9935, Val Loss: 0.0576, Val Accuracy: 0.9857
Epoch [6/30] - Train Loss: 0.0039, Train Accuracy: 0.9990, Val Loss: 0.0570, Val Accuracy: 0.9863
Epoch [7/30] - Train Loss: 0.0029, Train Accuracy: 0.9995, Val Loss: 0.0417, Val Accuracy: 0.9883
Epoch [8/30] - Train 

...

bundle: F0

Test Set Evaluation - Fold 2
Test Accuracy: 0.9947643979057592 , Precision: 0.9948566552113846, F1: 0.9947762715738488, Test Recall: 0.9947643979057592, error: 0.034443009101184
Test Confusion Matrix:
[[165   0   2   0]
 [  0 190   1   0]
 [  0   0 223   0]
 [  0   0   1 182]]
Epoch [1/30] - Train Loss: 0.7405, Train Accuracy: 0.9240, Val Loss: 0.0928, Val Accuracy: 0.9772
Epoch [2/30] - Train Loss: 0.0603, Train Accuracy: 0.9818, Val Loss: 0.1460, Val Accuracy: 0.9532
Epoch [3/30] - Train Loss: 0.0396, Train Accuracy: 0.9868, Val Loss: 0.0621, Val Accuracy: 0.9833
Epoch [4/30] - Train Loss: 0.0287, Train Accuracy: 0.9910, Val Loss: 0.0483, Val Accuracy: 0.9866
Epoch [5/30] - Train Loss: 0.0298, Train Accuracy: 0.9907, Val Loss: 0.0311, Val Accuracy: 0.9928
Epoch [6/30] - Train Loss: 0.0138, Train Accuracy: 0.9955, Val Loss: 0.0393, Val Accuracy: 0.9905
Epoch [7/30] - Train Loss: 0.0267, Train Accuracy: 0.9912, Val Loss: 0.0477, Val Accuracy: 0.9844
Epoch [8/30] - Train Los

/tmp/ipykernel_25415/3627583972.py:264: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_loader = DataLoader(TensorDataset(torch.tensor(testX_v), torch.tensor(testX_i), torch.tensor(test_feature), torch.tensor(testY)),


bundle: F0

Test Set Evaluation - Fold 3
Test Accuracy: 0.9973821989528796 , Precision: 0.9974054682955207, F1: 0.9973845315271501, Test Recall: 0.9973821989528796, error: 0.025730628153149894
Test Confusion Matrix:
[[167   0   0   0]
 [  0 190   1   0]
 [  0   0 223   0]
 [  0   0   1 182]]
Epoch [1/30] - Train Loss: 0.7167, Train Accuracy: 0.9305, Val Loss: 0.0855, Val Accuracy: 0.9794
Epoch [2/30] - Train Loss: 0.0549, Train Accuracy: 0.9832, Val Loss: 0.0689, Val Accuracy: 0.9833
Epoch [3/30] - Train Loss: 0.0376, Train Accuracy: 0.9891, Val Loss: 0.0588, Val Accuracy: 0.9850
Epoch [4/30] - Train Loss: 0.0215, Train Accuracy: 0.9933, Val Loss: 0.0727, Val Accuracy: 0.9749
Epoch [5/30] - Train Loss: 0.0206, Train Accuracy: 0.9939, Val Loss: 0.0711, Val Accuracy: 0.9783
Epoch [6/30] - Train Loss: 0.0300, Train Accuracy: 0.9901, Val Loss: 0.0765, Val Accuracy: 0.9822
Epoch [7/30] - Train Loss: 0.0144, Train Accuracy: 0.9948, Val Loss: 0.0806, Val Accuracy: 0.9788
Epoch [8/30] - Train 

/tmp/ipykernel_25415/3627583972.py:264: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_loader = DataLoader(TensorDataset(torch.tensor(testX_v), torch.tensor(testX_i), torch.tensor(test_feature), torch.tensor(testY)),


bundle: F0

Test Set Evaluation - Fold 4
Test Accuracy: 0.9921465968586387 , Precision: 0.9922619184906706, F1: 0.992149769950817, Test Recall: 0.9921465968586387, error: 0.035693616964899914
Test Confusion Matrix:
[[163   0   4   0]
 [  0 190   1   0]
 [  0   1 222   0]
 [  0   0   0 183]]
